In [92]:
from collections import deque

import numpy as np

In [93]:
moves2directions = {
    '^': (-1, 0), 
    'v': (1, 0), 
    '<': (0, -1), 
    '>': (0, 1)
}

In [94]:
def parse_input(file, part):
    with open(file) as file_in:
        input_str = file_in.read()

    if part == 2:
        input_str = input_str.replace('#', '##')
        input_str = input_str.replace('O', '[]')
        input_str = input_str.replace('.', '..')
        input_str = input_str.replace('@', '@.')

    grid, moves = input_str.split('\n\n')
    grid = np.array([list(row) for row in grid.splitlines()])
    moves = ''.join(moves.splitlines())

    return grid, moves

In [95]:
def get_new_pos_boxes_p1(x_current, y_current, move, pos_boxes, pos_walls):
    nx, ny = moves2directions[move]
    x_next, y_next = x_current + nx, y_current + ny

    boxes_to_move = set()
    while (x_next, y_next) in pos_boxes:
        boxes_to_move.add((x_next, y_next))
        x_next, y_next = x_next + nx, y_next + ny

    if (x_next, y_next) in pos_walls:
        return []
    else:
        pos_boxes = set([(x, y) if (x, y) not in boxes_to_move else (x+nx, y+ny) for x, y in pos_boxes])
        return pos_boxes

In [96]:
def get_final_boxes_pos(x_start, y_start, moves, pos_boxes, pos_walls):
    x_current, y_current = x_start, y_start
    for move in moves:
        nx, ny = moves2directions[move]
        x_next, y_next = x_current + nx, y_current + ny
        if (x_next, y_next) in pos_walls:
            continue
        elif (x_next, y_next) in pos_boxes:
            new_pos_boxes = get_new_pos_boxes_p1(x_current, y_current, move, pos_boxes, pos_walls)
            if new_pos_boxes:
                pos_boxes = new_pos_boxes
                x_current, y_current = x_next, y_next
        else:
            x_current, y_current = x_next, y_next

    return pos_boxes

In [97]:
def main1(file):
    grid, moves = parse_input(file, part=1)
    pos_boxes = set([(x, y) for x, y in np.argwhere(grid == 'O').tolist()])
    pos_walls = set([(x, y) for x, y in np.argwhere(grid == '#').tolist()])

    x_start, y_start = np.argwhere(grid == '@')[0]
    final_pos_boxes = get_final_boxes_pos(x_start, y_start, moves, pos_boxes, pos_walls)

    sum_gps = 0
    for x, y in final_pos_boxes:
        sum_gps += 100 * x + y
    
    return sum_gps

In [98]:
def get_parts_box(x_box, y_box, pos_boxes):
    pos_part1 = (x_box, y_box)
    pos_part2 = (x_box, y_box+1) if pos_boxes[x_box, y_box] == '[' else (x_box, y_box-1)
    return [pos_part1, pos_part2]

In [99]:
def get_new_pos_boxes_p2(x_current, y_current, move, pos_boxes, pos_walls):
    nx, ny = moves2directions[move]
    x_next, y_next = x_current + nx, y_current + ny
    x_next, y_next = x_next.item(), y_next.item()

    if move == '>' or move == '<':
        # Horizontal move
        boxes_to_move = set()
        while (x_next, y_next) in pos_boxes:
            boxes_to_move.add((x_next, y_next))
            x_next, y_next = x_next, y_next + ny

        if (x_next, y_next) in pos_walls:
            return []
        else:
            new_pos_boxes = {}
            for x, y in pos_boxes:
                if (x, y) in boxes_to_move:
                    new_pos_boxes[(x, y + ny)] = pos_boxes[(x, y)]
                else:
                    new_pos_boxes[(x, y)] = pos_boxes[(x, y)]
            return new_pos_boxes
    else:
        # Vertical move
        pos_box_init = get_parts_box(x_next, y_next, pos_boxes)

        # Use BFS to get the coordinates of all potentially impacted boxes
        pos_impacted_boxes = set(pos_box_init)
        queue = deque(pos_box_init)
        bfs_visited = set()
        while queue:
            x_box, y_box = queue.popleft()
            x_box_next, y_box_next = x_box + nx, y_box
            if (x_box_next, y_box_next) in pos_boxes and (x_box_next, y_box_next) not in bfs_visited:
                parts_next_box = get_parts_box(x_box_next, y_box_next, pos_boxes)
                pos_impacted_boxes.update(parts_next_box)
                bfs_visited.update(parts_next_box)
                queue.extend(parts_next_box)

        # If all boxes can be moved, move them
        pos_final_boxes = set([(x + nx, y) for x, y in pos_impacted_boxes])
        if pos_final_boxes.isdisjoint(pos_walls):
            new_pos_boxes = {}
            for x, y in pos_boxes:
                if (x, y) in pos_impacted_boxes:
                    new_pos_boxes[(x + nx, y)] = pos_boxes[(x, y)]
                else:
                    new_pos_boxes[(x, y)] = pos_boxes[(x, y)]
            return new_pos_boxes

In [100]:
def debug(grid, pos_walls, pos_boxes, x_current, y_current):
    plot = np.full_like(grid, '.')
    for x, y in pos_walls:
        plot[x, y] = '#'
    for x, y in pos_boxes:
        plot[x, y] = pos_boxes[(x, y)]
    plot[x_current, y_current] = '@'
    print(plot)
    print('\n\n')

In [101]:
def main2(file):
    grid, moves = parse_input(file, part=2)

    pos_boxes = [(x, y) for x, y in np.argwhere((grid == '[') | (grid == ']')).tolist()]
    pos_boxes = {(x, y): grid[x, y].item() for (x, y) in pos_boxes}
    pos_walls = set([(x, y) for x, y in np.argwhere(grid == '#').tolist()])
    x_start, y_start = np.argwhere(grid == '@')[0]

    x_current, y_current = x_start, y_start

    for move in moves:
        nx, ny = moves2directions[move]
        x_next, y_next = x_current + nx, y_current + ny
        if (x_next, y_next) in pos_walls:
            continue
        elif (x_next, y_next) in pos_boxes:
            new_pos_boxes = get_new_pos_boxes_p2(x_current, y_current, move, pos_boxes, pos_walls)
            if new_pos_boxes:
                pos_boxes = new_pos_boxes
                x_current, y_current = x_next, y_next
        else:
            x_current, y_current = x_next, y_next

    sum_gps = 0
    for x, y in pos_boxes:
        if pos_boxes[(x, y)] == '[':
            sum_gps += 100 * x + y

    return sum_gps

In [102]:
assert main1('example1.txt') == 2028
assert main1('example2.txt') == 10092

In [103]:
main1('input.txt')

1448589

In [77]:
main2('input.txt')

1472235